### Bi-LSTM
Bi-directional LSTM은 시퀀스를 순방향(forward)과 역방향(backward) 두 방향으로 처리하는 LSTM을 병렬적으로 구성하여, 과거와 미래의 문맥 정보를 동시에 반영할 수 있는 순환 신경망 구조이다. 이를 통해 시계열 데이터나 자연어 처리와 같이 양방향 문맥이 중요한 문제에서 보다 정교한 특징 표현이 가능하다.

각각 순방향 LSTM, 역방향 LSTM 하게 하여 각각을 수행한 후 그 결과들을 concatenate 시켜 예측에 이용한다.  


<img src="./images/bi-LSTM.png" width="400"/>

In [1]:
import torch
import torchvision
import torch.nn as nn 
import torch.optim as optim  
from torch.utils.data import DataLoader

In [ ]:
# Load Data
tensor_mode = torchvision.transforms.ToTensor()
trainset = torchvision.datasets.MNIST(root="./data", train=True, transform=tensor_mode, download=True) # 손글씨 이미지로 표현한 흑백 이미지(Grayscale) 데이터셋
testset = torchvision.datasets.MNIST(root="./data", train=False, transform=tensor_mode, download=True)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=False)

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.72MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 156kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.29MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.39MB/s]


In [3]:
class BiLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, seq_length, num_classes, device):
    super(BiLSTM, self).__init__()
    self.device = device
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.seq_length = seq_length
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
    self.fc = nn.Linear(seq_length*hidden_size * 2, num_classes) # classifier 에 해당하는 fully connected layer에 넣어준다.

  def forward(self, x):
    h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(self.device) # 순방향, 역방향을 고려하여 2개의 레이어가 필요하다.
    c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(self.device)
    out, _ = self.lstm(x, (h0, c0))
    #out = self.fc(out[:, -1, :])
    out = out.reshape(-1,self.seq_length*self.hidden_size * 2) # 배열을 1열로 만들어준다
    out = self.fc(out)
    return out

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sequence_length = trainset.data.size(1)
input_size = trainset.data.size(2)
num_layers = 2
hidden_size = 12
num_classes = 10

In [5]:
model = BiLSTM(input_size, hidden_size, num_layers, sequence_length, num_classes, device)
model = model.to(device)   

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-3)

In [7]:
for epoch in range(11):
    correct = 0
    total = 0
    for data in trainloader:
        optimizer.zero_grad()
        inputs, labels = data[0].to(device).squeeze(1), data[1].to(device)  # [128, 1, 28, 28] -> [128, 28, 28]
        outputs = model(inputs)
        loss = criterion(outputs, labels)        
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.detach(), 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('[%d] train acc: %.2f' %(epoch, 100*correct/total))     

[0] train acc: 91.57
[1] train acc: 97.45
[2] train acc: 98.16
[3] train acc: 98.46
[4] train acc: 98.72
[5] train acc: 98.80
[6] train acc: 99.00
[7] train acc: 99.09
[8] train acc: 99.14
[9] train acc: 99.24
[10] train acc: 99.31


In [ ]:
def accuracy(dataloader):
    correct = 0
    total = 0
    
    with torch.no_grad():
        model.eval()
        for data in dataloader:
            inputs, labels = data[0].to(device).squeeze(1), data[1].to(device)      
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)      
            correct += (predicted == labels).sum().item()

    acc = 100*correct/total
    model.train()
    return acc

In [9]:
train_acc = accuracy(trainloader)
test_acc = accuracy(testloader)
print("Train Acc: %.1f, Test Acc: %.1f" %(train_acc, test_acc))

Train Acc: 99.3, Test Acc: 98.5


이미지 처리에 있어서 CNN을 고수 하지 않아도 된다. 최근에는 CNN을 전혀 이용하지 않은 Vision transformer가 나오기도 했다. 어떤 문제를 생각할떄 정형화된 방법이 있지 않다는 것을 염두해 두는 것이 좋다.